# Vehicle Noise Classification (UK GOV dataset)

**Objective:** Predict whether a vehicle is **Quiet** or **Noisy** from engine and emission features in the UK Government fuel/emissions data.

## 1. Setup & Imports

Import core libraries and configure display options.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

pd.set_option('display.max_columns', None)
sns.set_theme()
print("Environment ready.")